In [1]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from utils.data_cleaning import replace_chars

In [2]:
df = pd.read_csv('data/relations.csv')
df

,Individuo A: String,Detalle Relación: String,Relación: String,Individuo B: String,mencionadoEnEscandalo: String,Noticia URL: String,Organización A: String,IndividuoA_Tipo,IndividuoB_Tipo,Categorización_Relación
0,juan_camilo_salazar,manifestó inconformidad,manifestadoPor,fabio_espitia,AgroIngreso Seguro,https://www.portafolio.co/economia/finanzas/di...,NaN,PER,PER,Relación no reconocida
1,juan_camilo_salazar,deberá pagar una multa de más de 80 millones d...,pagadoPor,juan_camilo_salazar,AgroIngreso Seguro,https://www.portafolio.co/economia/finanzas/di...,NaN,PER,PER,TratosFinancieros
2,juan_camilo_salazar,será inhabilitado por los mismos años en los q...,inhabilitadoPor,juan_camilo_salazar,AgroIngreso Seguro,https://www.portafolio.co/economia/finanzas/di...,NaN,PER,PER,relacionesDeInvestigaciÃ³n
3,andres_felipe_arias,condena a Salazar,condenaA,juan_camilo_salazar,AgroIngreso Seguro,https://www.portafolio.co/economia/finanzas/di...,NaN,PER,PER,Relación no reconocida
4,andres_felipe_arias,aceptados por Salazar y por los que el 21 de o...,aceptadoPor,juan_camilo_salazar,AgroIngreso Seguro,https://www.portafolio.co/economia/finanzas/di...,NaN,PER,PER,Relación no reconocida
...,...,...,...,...,...,...,...,...,...,...
438,guillermo_leon_acevedo_giraldo,NaN,aliasDe,memo_fantasma,Insight Crime,https://es.insightcrime.org/en-video-todo-sobr...,Memo Fantasma,PER,PER,Relación no reconocida
439,memo_fantasma,NaN,aliasDe,guillermo_leon_acevedo_giraldo,Insight Crime,https://es.insightcrime.org/en-video-todo-sobr...,Guillermo León Acevedo Giraldo,PER,PER,Relación no reconocida
440,catalina_mejia_acosta,NaN,parejaDe,guillermo_leon_acevedo,Insight Crime,https://es.insightcrime.org/esposa-memo-fantas...,Guillermo León Acevedo,PER,PER,Relación no reconocida
441,catalina_mejia_acosta,NaN,madreDe,hijos_mas_pequenos_de_guillermo_leon_acevedo,Insight Crime,https://es.insightcrime.org/esposa-memo-fantas...,hijos más pequeños de Guillermo León Acevedo,PER,PER,Relación no reconocida


In [9]:
def categorize_relations(my_input):

    TEMPLATE = """
    Las siguientes son categorias que se pueden usar para clasificar las relaciones entre personas.
    **Relaciones Familiares**: Este tipo de relaciones describe los lazos familiares entre dos personas. Las relaciones familiares pueden indicar conexiones sanguíneas, como "hermano de", "hijo de", "madre de", o también pueden representar relaciones más complejas o extendidas como "primo hermano de", "exnovio de". Estas relaciones son cruciales para entender la genealogía y los lazos familiares.

    **Relaciones Laborales o Profesionales**: Estas relaciones son aquellas que indican un vínculo laboral o profesional entre dos entidades. Pueden variar desde roles jerárquicos en una organización (por ejemplo, "presidente de", "gerente de") hasta roles funcionales (por ejemplo, "abogado de", "representante de"). Estas relaciones ayudan a comprender la dinámica laboral, los roles y responsabilidades, y las conexiones profesionales entre individuos o entidades.

    **Relaciones de Corrupción**: Estas son relaciones que implican actos de corrupción o ilegalidad entre dos o más entidades. Estas relaciones pueden implicar delitos como sobornos ("pagado por"), inhabilitación ("inhabilitado por"), aceptación de cargos ("aceptó cargos"), entre otros. Estas relaciones ayudan a desvelar actos de corrupción y proporcionan un contexto para entender la naturaleza de dichos actos.

    **Relaciones de Investigación**: Estas relaciones indican la existencia de una investigación o proceso judicial en curso. Las frases como "investigado por", "acusado de", "condenado a" caen en esta categoría. Estas relaciones proporcionan información sobre el estado actual de las investigaciones y los procesos judiciales en curso.

    **Relaciones de Influencia**: Estas relaciones representan algún tipo de influencia, poder o control que una entidad ejerce sobre otra. Las frases como "es acusado de", "está detenido en", "trabajó para" se incluyen en esta categoría. Estas relaciones ayudan a identificar la dinámica de poder y las relaciones de influencia que podrían existir entre las partes.

    Para las siguientes relaciones en un tabla, añade dos columnas a la tabla, una que permita clasificar cada una de las relaciones de acuerdo a las categorías anteriores, y otra que permita añadir una descripción de la relación. Por ejemplo, si la relación es "es el padre de", la categoría sería "Relaciones Familiares" y la descripción podría ser "Es el padre de Juan Pérez".

    Output esperado:
    Detalle Relación|Relación|Categoría|Descripción

    Tabla a procesar:
    {relations}
    """

    PROMPT_NER = PromptTemplate(
        input_variables=["relations"],
        template=TEMPLATE,
        # partial_variables={"format_instructions": parserNER.get_format_instructions()},
    )

    ## Definición de la Cadena
    llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0.2, max_tokens=8000)

    llm_chain_NER = LLMChain(
        llm=llm,
        prompt=PROMPT_NER,
        # output_parse_function=parse_output,
    )

    chat_completion_NER = llm_chain_NER(my_input, return_only_outputs=True)

    return chat_completion_NER

In [4]:
from utils.data_cleaning import replace_chars

char_dict = {
    ":": "_",
    "á": "a",
    "é": "e",
    "í": "i",
    "ì": "i",
    "ó": "o",
    "ú": "u",
    "ñ": "n",
    "Á": "A",
    "É": "E",
    "Í": "I",
    "Ó": "O",
    "Ú": "U",
    "Ñ": "N",
    "¿": "",
    "?": "",
    "¡": "",
    "!": "",
    "(": "",
    ")": "",
    '"': "",
    "'": "",
    "“": "",
    "”": "",
    " ": "_",
}


In [10]:
chunk = 30
df_list = []
# replace_chars
for i in range(0, df.shape[0], chunk):
    print(i)
    df_split = df[['Detalle Relación: String', 'Relación: String']].iloc[i:i+chunk]
    text = df_split.to_string(index=False)
    response = categorize_relations(text)
    with open(f'data/relations_categorized_{i}.csv', 'w', encoding='utf-8') as f:
        f.write(response['text'])

    df_list.append(pd.read_csv(f'data/relations_categorized_{i}.csv', encoding='utf-8', sep='|', dtype=str))


0
30
60
90
120
150
180
210
240
270
300
330
360
390
420


In [11]:
df_final = pd.concat(df_list)
df_final

,Detalle Relación,Relación,Categoría,Descripción
0,---,---,---,---
1,manifestó inconformidad,manifestadoPor,Relaciones de Influencia,La persona manifestó su inconformidad
2,deberá pagar una multa de más de 80 millones d...,pagadoPor,Relaciones de Corrupción,La persona deberá pagar una multa de más de 80...
3,será inhabilitado por los mismos años en los q...,inhabilitadoPor,Relaciones de Corrupción,La persona será inhabilitada por los mismos añ...
4,condena a Salazar,condenaA,Relaciones de Investigación,La persona condena a Salazar
...,...,...,...,...
19,NaN,aliasDe,Relaciones de Investigación,Es un alias de alguien
20,NaN,aliasDe,Relaciones de Investigación,Es un alias de alguien
21,NaN,parejaDe,Relaciones Familiares,Es pareja de alguien
22,NaN,madreDe,Relaciones Familiares,Es madre de alguien


In [11]:
df_final = pd.concat(df_list)
df_final.columns = [replace_chars(i, char_dict) for i in list(df_final.columns)]
df_final

,"Detalle_Relacion,Relacion,Categoria,Descripcion"
0,"manifestó inconformidad,manifestadoPor,Relacio..."
1,deberá pagar una multa de más de 80 millones d...
2,será inhabilitado por los mismos años en los q...
3,"condena a Salazar,condenaA,Relaciones de Inves..."
4,aceptados por Salazar y por los que el 21 de o...
...,...
18,"NaN,aliasDe,Relaciones de..."
19,"NaN,aliasDe,Relaciones de..."
20,"NaN,parejaDe,Relaciones F..."
21,"NaN,madreDe,Relaciones Fa..."


In [87]:
df_final = pd.concat(df_list)
df_final.columns = [i.replace(" ", "") for i in list(df_final.columns)]

df_final['Relación'] = df_final['Relación'].astype(str)
df_final['Categoría'] = df_final['Categoría'].astype(str)
df_final.columns = [i.replace(" ", "") for i in list(df_final.columns)]
df_final['Relación'] = df_final['Relación'].apply(lambda x: x.replace(" ", ""))

KeyError: 'Relación'

In [65]:
def clean_char(my_string):
    if isinstance(my_string, str):
        my_string = my_string.replace(" ", "")
    else :
        my_string = ""
    return my_string


In [81]:
df_final = pd.concat(df_list)
df_final.columns = [i.replace(" ", "") for i in list(df_final.columns)]
df_final['Relación'] = df_final['Relación'].astype(str)
df_final['Categoría'] = df_final['Categoría'].astype(str)
df_final

KeyError: 'Relación'

In [18]:
df_final = pd.concat(df_list)
df_final.columns = [i.replace(" ", "") for i in list(df_final.columns)]
df_final['Relación'] = df_final['Relación'].astype(str)
df_final['Categoría'] = df_final['Categoría'].astype(str)
#df_final[['Relación', 'Categoría']] = df_final[['Relación', 'Categoría']].apply(clean_char)

resultado = {}

for _, row in df_final.iterrows():
    print(row['Categoría'])
    # Si la categoría ya existe en el diccionario, agregamos la nueva relación
    if row['Categoría'] in resultado.keys():
        resultado[row['Categoría']].append(row['Relación'])
    # Si la categoría no existe en el diccionario, la agregamos y asignamos la relación correspondiente
    else:
        resultado[row['Categoría']] = [row['Relación']]

print(resultado)

---
Relaciones de Influencia
Relaciones de Corrupción
Relaciones de Corrupción
Relaciones de Investigación
Relaciones de Corrupción
Relaciones Familiares
Relaciones de Corrupción
Relaciones Familiares
Relaciones Familiares
Relaciones Familiares
Relaciones Familiares
Relaciones de Investigación
Relaciones de Investigación
Relaciones de Investigación
Relaciones de Investigación
Relaciones de Investigación
Relaciones de Investigación
Relaciones de Investigación
Relaciones de Investigación
Relaciones de Investigación
Relaciones de Investigación
Relaciones de Investigación
Relaciones de Investigación
Relaciones de Investigación
Relaciones de Investigación
Relaciones de Investigación
Relaciones de Investigación
Relaciones de Investigación
Relaciones de Investigación
Relaciones de Corrupción
-
Relaciones de Investigación
Relaciones de Influencia
Relaciones de Corrupción
Relaciones de Corrupción
Relaciones de Corrupción
Relaciones de Corrupción
Relaciones de Influencia
Relaciones de Influencia

In [21]:
import json
with open('data/relations_categorized.json', 'w', encoding='utf-8') as f:
    json.dump(resultado, f, ensure_ascii=False, indent=4)